In [ ]:
import os
from os import walk
import types

from pylab import rcParams

import math
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
def cm2inch(value):
    return value/2.54

In [ ]:
len10min = 52417
len30min = 17473

In [ ]:
dataFiles = []

data_dir = '../data/unziped/'

for (dirpath, dirnames, filenames) in walk(data_dir):
    dataFiles.extend(filenames)
    break

data = []
names = []

for file in dataFiles:
    names.append(file[28:38])
    fname = os.path.join(data_dir, file)
    f = open(fname)
    nacitane = f.read()
    lines = nacitane.split(',')
    lines = np.array(lines)
    for i in range(len(lines)):
        if lines[i] == "None":
            lines[i] = "nan"
    lines = lines.astype('float32')

    data.append(lines)
    print(len(lines))
    f.close()

print(names)
print(len(data))

In [ ]:
df10 = pd.DataFrame() 
for i in range(len(data)):
    if names[i][0] == "1":
        df10.insert(len(df10.columns), names[i], data[i], True)
df10 = df10.reindex(sorted(df10.columns), axis=1)
df10

In [ ]:
df30 = pd.DataFrame() 
for i in range(len(data)):
    if names[i][0] == "3":
        df30.insert(len(df30.columns), names[i], data[i], True)
df30 = df30.reindex(sorted(df30.columns), axis=1)
df30

In [ ]:
diery = []

for item in data:
    dieryInItem = []
    for i in range(len(item)-1):
        if math.isnan(item[i]):
            dieryInItem.append(i)
            if not math.isnan(item[i+1]):
                diery.append(dieryInItem)
                dieryInItem = []

print(len(diery))
# print(diery)


In [22]:

x = range(len10min)

fig, ax = plt.subplots(22)
fig.set_size_inches(cm2inch(100), cm2inch(200), forward=True)

for i in range(22):
    ax[i].plot(x, df10[df10.columns[i]])
fig.savefig('test2png.png', dpi=500)

ValueError: Image size of 39370x78740 pixels is too large. It must be less than 2^16 in each direction.

In [ ]:
x = range(len30min)

fig, ax = plt.subplots(22)

for i in range(22):
    ax[i].plot(x, df30[df30.columns[i]])
fig.savefig('test2png.png', dpi=200)

In [ ]:
for i in range(1, 7):
    plt.subplot(2, 3, i)
    plt.text(0.5, 0.5, str((2, 3, i)),
             fontsize=18, ha='center')

In [ ]:
print("ahoj")

In [ ]:
import numpy as np
float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values


In [ ]:
from matplotlib import pyplot as plt
temp = float_data[:, 1] 
plt.plot(range(len(temp)), temp)


In [ ]:
plt.plot(range(1440), temp[:1440])


In [ ]:
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std


In [ ]:
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
                rows = np.arange(i, min(i + batch_size, max_index))
                i += len(rows)
        samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets


In [ ]:
a = np.array(range(200)).reshape(100,2)
# print(a)
# print(float_data.shape)
print(next(generator(a, 
               lookback=10, 
               delay=0, 
               min_index=0, 
               max_index=20,
               shuffle=True, 
               step=1,
               batch_size=1)))

In [ ]:
lookback = 1440
step = 6
delay = 144
batch_size = 128

In [ ]:
train_gen = generator(float_data, 
                      lookback=lookback, 
                      delay=delay, 
                      min_index=0, 
                      max_index=200000,
                      shuffle=True, 
                      step=step, 
                      batch_size=batch_size)
val_gen = generator(float_data, 
                    lookback=lookback, 
                    delay=delay, 
                    min_index=200001,
                    max_index=300000, 
                    step=step, 
                    batch_size=batch_size)
test_gen = generator(float_data, 
                     lookback=lookback, 
                     delay=delay, 
                     min_index=300001,
                     max_index=None, 
                     step=step, 
                     batch_size=batch_size)
val_steps = (300000 - 200001 - lookback)
test_steps = (len(float_data) - 300001 - lookback)

In [ ]:
next(val_gen)

In [ ]:
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))

evaluate_naive_method()


In [ ]:
//////////